In [1]:
import numpy as np

def check_victory(board):
    """
    Vérifie si un joueur a gagné sur le plateau 3x3.
    :param board: Une matrice NumPy 3x3 contenant 1, -1 et 0.
    :return: 1 si Joueur A gagne, -1 si Joueur B gagne, 0 sinon.
    """
    # Vérifier les lignes et les colonnes
    for i in range(3):
        if abs(sum(board[i, :])) == 3:  # Ligne complète
            return np.sign(board[i, 0])
        if abs(sum(board[:, i])) == 3:  # Colonne complète
            return np.sign(board[0, i])

    # Vérifier les diagonales
    if abs(board[0, 0] + board[1, 1] + board[2, 2]) == 3:
        return np.sign(board[0, 0])
    if abs(board[0, 2] + board[1, 1] + board[2, 0]) == 3:
        return np.sign(board[0, 2])

    return 0  # Pas de victoire

# Exemple de test
board = np.array([
    [1, 1, 1],
    [-1, 0, -1],
    [0, -1, 0]
])
print(f"Résultat : {check_victory(board)}")  # Doit afficher 1 (Joueur A gagne)


Résultat : 1


In [13]:
import numpy as np
import random

def check_victory(board):
    """Vérifie si le joueur 1 ou -1 a gagné."""
    for i in range(3):
        # Vérifier les lignes et colonnes
        if np.all(board[i, :] == 1) or np.all(board[:, i] == 1):
            return 1  # Joueur 1 gagne
        if np.all(board[i, :] == -1) or np.all(board[:, i] == -1):
            return -1  # Joueur -1 gagne

    # Vérifier les diagonales
    if np.all(np.diag(board) == 1) or np.all(np.diag(np.fliplr(board)) == 1):
        return 1
    if np.all(np.diag(board) == -1) or np.all(np.diag(np.fliplr(board)) == -1):
        return -1

    return 0  # Pas de gagnant

def generate_random_board():
    """Génère un plateau 3x3 valide avec exactement 3 pièces par joueur."""
    board = np.zeros((3, 3), dtype=int)
    positions = random.sample(range(9), 6)  # 6 positions uniques

    # Placer 3 pièces du Joueur 1
    for i in range(3):
        board[positions[i] // 3, positions[i] % 3] = 1

    # Placer 3 pièces du Joueur -1
    for i in range(3, 6):
        board[positions[i] // 3, positions[i] % 3] = -1

    return board

def generate_dataset():
    """Génère 500 positions gagnantes et 500 positions perdantes pour le joueur 1."""
    X, y = [], []
    win_count, lose_count = 0, 0

    while win_count < 500 or lose_count < 500:
        board = generate_random_board()
        result = check_victory(board)

        if result == 1 and win_count < 500:
            X.append(board.flatten())
            y.append(1)
            win_count += 1
        elif result == -1 and lose_count < 500:
            X.append(board.flatten())
            y.append(-1)
            lose_count += 1

    return np.array(X), np.array(y)

# Générer le dataset
X, y = generate_dataset()

# Vérification
print(f"Nombre d'échantillons : {X.shape[0]}")
print(f"Exemple de position gagnante : \n{X[0].reshape(3, 3)} -> {y[0]}")
print(f"Exemple de position perdante : \n{X[500].reshape(3, 3)} -> {y[500]}")

Nombre d'échantillons : 1000
Exemple de position gagnante : 
[[-1 -1 -1]
 [ 0  1  1]
 [ 0  0  1]] -> -1
Exemple de position perdante : 
[[ 0 -1  1]
 [ 0  0  1]
 [-1 -1  1]] -> 1


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Séparation en données d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entraîner un modèle de régression linéaire
model = LinearRegression()
model.fit(X_train, y_train)

# Évaluation du modèle
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Erreur quadratique moyenne (MSE) : {mse}")


Erreur quadratique moyenne (MSE) : 0.18081950650980713


In [10]:
import tensorflow as tf
from tensorflow import keras

# Définir le modèle MLP
model = keras.Sequential([
    keras.layers.Dense(32, activation='relu', input_shape=(9,)),  # 9 entrées (3x3)
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='tanh')  # Sortie entre -1 et 1
])

# Compilation du modèle
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Entraînement du modèle
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Évaluation finale
loss, mae = model.evaluate(X_test, y_test)
print(f"Perte (Loss) : {loss}, Erreur absolue moyenne (MAE) : {mae}")


Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


125/125 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - loss: 0.2190 - mae: 0.3056 - val_loss: 0.1732 - val_mae: 0.2438
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1588 - mae: 0.2333 - val_loss: 0.1556 - val_mae: 0.2354
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.1378 - mae: 0.2206 - val_loss: 0.1407 - val_mae: 0.2301
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1243 - mae: 0.2207 - val_loss: 0.1238 - val_mae: 0.2239
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - loss: 0.1219 - mae: 0.2305 - val_loss: 0.1141 - val_mae: 0.2248
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.1049 - mae: 0.2211 - val_loss: 0.1048 - val_mae: 0.2252
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0998 - mae: 0.2252 - val_loss: 0.0986 - val_mae: 0.2261
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0909 - mae: 0.2193 - val_loss: 0.0893 - val_mae: 0.2162
Epoch 9/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0913 

In [11]:
# Exemple d'une position
test_board = np.array([
    [1, 0, -1],
    [-1, 1, 0],
    [0, -1, 1]
])

# Transformer en vecteur 1D
test_input = test_board.flatten().reshape(1, -1)

# Prédiction du modèle
predicted_value = model.predict(test_input)[0][0]
print(f"Évaluation prédite pour cette position : {predicted_value}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
Évaluation prédite pour cette position : 0.8108914494514465


In [12]:
for _ in range(5):
    board = generate_random_board()
    evaluation = model.predict(board.flatten().reshape(1, -1))[0][0]
    print(f"Position : \n{board}")
    print(f"Évaluation prédite : {evaluation}\n")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Position : 
[[ 1  0 -1]
 [ 1 -1  0]
 [ 1  0 -1]]
Évaluation prédite : 0.871063768863678

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Position : 
[[ 1 -1  0]
 [ 0 -1 -1]
 [ 0  1  1]]
Évaluation prédite : 0.08841951936483383

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Position : 
[[-1  1  1]
 [ 0 -1  0]
 [-1  0  1]]
Évaluation prédite : 0.2478589415550232

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
Position : 
[[-1  0 -1]
 [-1  0  0]
 [ 1  1  1]]
Évaluation prédite : 0.7010610103607178

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Position : 
[[ 1 -1 -1]
 [ 1 -1  0]
 [ 0  1  0]]
Évaluation prédite : 0.2579823136329651

